In [2]:
import pandas as pd
import numpy as np

## Preprocess **Kon Plong image metadata spreadsheet**

In [ ]:
#@title Full spreadsheet

full_metadata = pd.read_csv('image_metadata(2020-06-26)_full.csv')
full_metadata.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,20,41,42,45,56,57,58,60,61,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SourceFile,BitsPerSample,Caption-Abstract,CatalogSets,Categories,CodedCharacterSet,ColorComponents,ColorLabel,Comment,CreateDate,CreatorTool,CurrentIPTCDigest,DateCreated,DateTime,DateTimeCreated,DateTimeOriginal,DateUTC,Description,Directory,EncodingProcess,Error,ExifByteOrder,ExifImageHeight,ExifImageWidth,ExifToolVersion,ExifVersion,FileAccessDate,FileCreateDate,FileModifyDate,FileName,FilePermissions,FileSize,FileType,FileTypeExtension,Flash,HierarchicalSubject,ImageDescription,ImageHeight,ImageNumber,ImageSize,ImageWidth,InstanceID,IPTCDigest,JFIFVersion,Keywords,LastKeywordIPTC,LastKeywordXMP,Make,Megapixels,MetadataDate,MIMEType,Model,ModificationDate,ModifyDate,Notes,OffsetSchema,Orientation,OriginatingProgram,Padding,PickLabel,ProcessingSoftware,ProgramVersion,Rating,RatingPercent,RegionInfoRegions,RegionList,ResolutionUnit,Software,Subject,TagsList,TimeCreated,Urgency,UserComment,Warning,XMPToolkit,XPKeywords,XResolution,YCbCrSubSampling,YResolution
0,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:09:11,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:09:11,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:05:26 10:18:20+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-09-11(1).JPG,rw-rw-rw-,458 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1154.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:09:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
1,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:09:42,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:09:42,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:06:24 14:00:28+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-09-42(2).JPG,rw-rw-rw-,471 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1155.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:09:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
2,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:12:32,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:12:32,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:06:24 14:00:29+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-12-32(1).JPG,rw-rw-rw-,485 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1155.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:12:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
3,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Se

In [ ]:
#@title Keep important columns and Save to metadata.csv

metadata = full_metadata[['SourceFile', 'Directory', 'FileName', 'DateTimeOriginal', 'CatalogSets', 'Make', 'Model']]

# Save to file
metadata.to_csv('metadata.csv', index = False)

metadata.head()

,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096


In [3]:
#@title Open metadata.csv
metadata = pd.read_csv('metadata.csv')

metadata.head()

,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096


Create new columns:
- **certainty**: an id is certain or not, follow WI guideline. Values = ['Absolutely sure', 'Not  sure']
- **species_common_name**: extract from tag **Species** in **CatalogSets**.
- **observer**: extract from tag **Observer** in **CatalogSets**.
- **people**: specify if this image has human
- **select**: specify if this image has already been uploaded to WI
- **station**: the station of the camera
- **camera_id**: the camera that took this image

In [11]:
#@title Remove redundant + Create new columns and save to metadata_new_column.csv

metadata_new_column = metadata.copy()

certainty = [] 
species_common_name = [] 
observer = []
people = []
select = []
station = []
camera_id = []

UNCERTAIN = 'Uncertain Identification'
SPECIES_PREFIX = 'Species'
OBSERVER_PREFIX = 'Observer'
PEOPLE_PREFIX = 'People'

# Create certainty, species_common_name, observer, and people columns
catalog_sets = metadata['CatalogSets']
for i in range(len(catalog_sets)):
  select.append('')
  if catalog_sets[i] is not np.NaN:
    catalog = str(catalog_sets[i]).split(', ')
    if UNCERTAIN in catalog:
      certainty.append('Not sure')
    else:
      certainty.append('Absolutely sure')
    spe = ''
    obs = ''
    peo = ''
    for i in range(len(catalog)):
      if SPECIES_PREFIX in catalog[i]:
        if catalog[i].split('|')[1] != 'Human':
          if '(unknown)' in catalog[i].split('|')[1]:
            spe = spe + ', ' + catalog[i].split('|')[1].split('(unknown)')[0]
          else:
            spe = spe + ', ' + catalog[i].split('|')[1]
      if OBSERVER_PREFIX in catalog[i]:
        obs = obs + ', ' + catalog[i].split('|')[1]
      if PEOPLE_PREFIX in catalog[i]:
        if not 'Human' in spe:
          spe = spe + ', ' + 'Human'
        peo = peo + ', ' + catalog[i]
    species_common_name.append(spe[2:])
    observer.append(obs[2:])
    people.append(peo[2:])

  else:
    certainty.append('')
    observer.append('')
    species_common_name.append('')
    people.append('')

# Create station and camera_id column
for file in metadata_new_column['FileName'].to_list():
  station.append(file.split('__')[0])
  camera_id.append(file.split('__')[1])

metadata_new_column = metadata.copy()
metadata_new_column['certainty'] = certainty
metadata_new_column['species_common_name'] = species_common_name
metadata_new_column['observer'] = observer
metadata_new_column['people'] = people
metadata_new_column['select'] = select
metadata_new_column['station'] = station
metadata_new_column['camera_id'] = camera_id

# Save to file
metadata_new_column.to_csv('metadata_new_column.csv', index = False)

print('Saved file to metadata_new_column.csv')

metadata_new_column.head(5)

Saved file to metadata_new_column.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,species_common_name,observer,people,select,station,camera_id
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,,100,67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,,100,67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,,100,67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,,100,67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,,100,67096


In [12]:
#@title Open metadata_new_column.csv
metadata_new_column = pd.read_csv('metadata_new_column.csv')

metadata_new_column.head()

,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,species_common_name,observer,people,select,station,camera_id
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,NaN,100,67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,NaN,100,67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,NaN,100,67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,NaN,100,67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,NaN,100,67096


[Could be ignored] Select images have been uploaded before, i.e. folders **236**, **237**, and **100/67096** through manual upload, and 85 Owston's civet images of folder **275**.

### To do this, upload **images_275_Owston.csv**

In [14]:
#@title Select images in folder 236+237+100/67096 and Save to metadata_select.csv

def select_images(metadata_select):
  # Select images in folder 236+237+100/67096
  select = []

  for file in metadata_select['FileName'].to_list():
    if (file.split('__')[0] == '236') or (file.split('__')[0] == '237'):
      select.append('x')
    elif (file.split('__')[0] == '100') and (file.split('__')[1] == '67096'):
      select.append('x')
    else:
      select.append('')
  metadata_select['select'] = select

  # Select 85 images uploaded
  img_data = pd.read_csv('images_275_Owston.csv')

  metadata_select.loc[metadata_select['FileName'].isin(img_data['image_id']), 'select'] = 'x'
  return metadata_select


metadata_select = metadata_new_column.copy()

metadata_select = select_images(metadata_select)

# Check the number of images uploaded
print('Total:',len(metadata_select), 'images')
print('Uploaded:', len(metadata_select[metadata_select['select'] == 'x']), 'images')

# Save to file
metadata_select.to_csv('metadata_select.csv', index = False)
print('Saved to metadata_select.csv')

Total: 110235 images
Uploaded: 2945 images
Saved to metadata_select.csv


In [15]:
#@title Open metadata_select.csv
metadata_select = pd.read_csv('metadata_select.csv')

print('Read metadata_select.csv')

metadata_select.head()

Read metadata_select.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,species_common_name,observer,people,select,station,camera_id
0,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-11(1).JPG,2019:04:22 15:09:11,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,x,100,67096
1,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-09-42(2).JPG,2019:04:22 15:09:42,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,x,100,67096
2,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-32(1).JPG,2019:04:22 15:12:32,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,x,100,67096
3,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-12-55(2).JPG,2019:04:22 15:12:55,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,x,100,67096
4,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__67096__2019-04-22__15-13-18(1).JPG,2019:04:22 15:13:18,"People|Setup, Observer|Tam",Panthera V4,CAM67096,Absolutely sure,Human,Tam,People|Setup,x,100,67096


In [16]:
# @title Spilt a row with multiple species into multiple rows, each contains one species & Save to split_species_column.csv 
"""
Pythonic steps:
1. Set the columns not to be touched as the index,
2. The next step is a 2-step process: Split on comma to get a column of lists, 
then call explode to explode the list values into their own rows
3. Finally, reset the index.
"""

col = metadata_select.columns.to_list()
col.remove('species_common_name')

split_species_column = metadata_select.copy()
split_species_column = split_species_column.set_index(col).apply(lambda x: x.str.split(',').explode()).reset_index()
split_species_column['species_common_name'] = split_species_column['species_common_name'].str.strip()

split_species_column.to_csv('split_species_column.csv', index = False)
print('Saved file to split_species_column.csv')

split_species_column.dropna(subset=['species_common_name'], inplace = False).sample(5)

Saved file to split_species_column.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,observer,people,select,station,camera_id,species_common_name
49817,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,280__67163__2019-12-26__09-13-11(2).JPG,2019:12:26 09:13:11,"Observer|Tam, Species|Pallas's Squirrel",Panthera V4,CAM67163,Absolutely sure,Tam,NaN,NaN,280,67163,Pallas's Squirrel
42207,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc259__ct67014__2019-10-28__10-29-47(3)__Emer...,2019:10:28 10:29:47,"Species|Emerald Dove, Observer|An Nguyen",Panthera V4,CAM67014,Absolutely sure,An Nguyen,NaN,NaN,kpc259,ct67014,Emerald Dove
100230,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc434__ct68988__2019-11-28__19-57-12(1)__Muri...,2019:11:28 19:57:12,"Observer|An Nguyen, Species|Murid (unknown)",Panthera V4,CAM68988,Absolutely sure,An Nguyen,NaN,NaN,kpc434,ct68988,Murid
62569,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc315__ct68912__2019-11-16__13-00-03(1)__Unid...,2019:11:16 13:00:03,"Species|Bird (unknown), Observer|An Nguyen",Panthera V4,CAM68912,Absolutely sure,An Nguyen,NaN,NaN,kpc315,ct68912,Bird
31743,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc239__ct69010__2019-12-07__11-09-18(1)__Unid...,2019:12:07 11:09:18,"Species|Bird (unknown), Observer|An Nguyen",Panthera V4,CAM69010,Absolutely sure,An Nguyen,NaN,NaN,kpc239,ct69010,Bird


In [ ]:
#@title Open split_species_column.csv
split_species_column = pd.read_csv('split_species_column.csv'
print('Read split_species_column.csv')
split_species_column.dropna(subset=['species_common_name'], inplace = False).sample(5)

Read split_species_column.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,observer,people,select,station,camera_id,species_common_name
19903,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc222__ct68861__2019-11-02__21-39-37(1)__Muri...,2019:11:02 21:39:37,"Observer|An Nguyen, Species|Murid (unknown)",Panthera V4,CAM68861,Absolutely sure,An Nguyen,NaN,NaN,kpc222,ct68861,Murid
30774,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc239__ct69010__2019-11-14__06-00-20(1)__Unid...,2019:11:14 06:00:20,"Species|Bird (unknown), Observer|An Nguyen",Panthera V4,CAM69010,Absolutely sure,An Nguyen,NaN,NaN,kpc239,ct69010,Bird
51476,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,292__66990__2019-04-27__10-35-34(3).JPG,2019:04:27 10:35:34,"Observer|Tam, Species|Rufous-throated Partridge",Panthera V4,CAM66990,Absolutely sure,Tam,NaN,NaN,292,66990,Rufous-throated Partridge
93436,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,397__68824__2019-11-23__07-06-40(2).JPG,2019:11:23 07:06:40,"Species|Orange-headed Thrush, Observer|Ha Anh",Panthera V4,CAM68824,Absolutely sure,Ha Anh,NaN,NaN,397,68824,Orange-headed Thrush
1723,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,100__68952__2019-06-22__18-21-26(2).JPG,2019:06:22 18:21:26,"Observer|Tam, Species|Blue Whistling-thrush",Panthera V4,CAM68952,Absolutely sure,Tam,NaN,NaN,100,68952,Blue Whistling-thrush


In [17]:
#@title Remove blank images & Save to metadata_remove_blank.csv

metadata_remove_blank = split_species_column.copy()
metadata_remove_blank['species_common_name'].replace('', np.nan, inplace=True)
metadata_remove_blank['people'].replace('', np.nan, inplace=True)
metadata_remove_blank.dropna(subset=['people', 'species_common_name'], how = 'all', inplace=True)

# Save to file
metadata_remove_blank.to_csv('metadata_remove_blank.csv', index = False)
print('Saved file to metadata_remove_blank.csv')

Saved file to metadata_remove_blank.csv


In [ ]:
#@title Open metadata_remove_blank.csv
metadata_remove_blank = pd.read_csv("metadata_remove_blank.csv")
print('Read metadata_remove_blank.csv')
metadata_remove_blank.sample(5)

Read metadata_remove_blank.csv


,SourceFile,Directory,FileName,DateTimeOriginal,CatalogSets,Make,Model,certainty,observer,people,select,station,camera_id,species_common_name
22219,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc386__ct67170__2019-11-27__18-29-00(1)__Muri...,2019:11:27 18:29:00,"Observer|An Nguyen, Species|Murid (unknown)",Panthera V4,CAM67170,Absolutely sure,An Nguyen,NaN,NaN,kpc386,ct67170,Murid
19671,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,357__68916__2019-10-30__02-25-37(1).JPG,2019:10:30 02:25:37,"Observer|Tam, Species|Ferret-badger (unknown)",Panthera V4,CAM68916,Absolutely sure,Tam,NaN,NaN,357,68916,Ferret-badger
1778,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,158__68823__2019-05-05__13-11-00(1).JPG,2019:05:05 13:11:00,"Observer|Tam, Species|Emerald Dove",Panthera V4,CAM68823,Absolutely sure,Tam,NaN,NaN,158,68823,Emerald Dove
21583,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,374__66941__2019-12-16__12-50-40(5).JPG,2019:12:16 12:50:40,"Observer|Tam, Species|Northern Treeshrew",Panthera V4,CAM66941,Absolutely sure,Tam,NaN,NaN,374,66941,Northern Treeshrew
6305,D:/Fauna & Flora International/VietnamCameraSt...,D:/Fauna & Flora International/VietnamCameraSt...,kpc240__ct66927__2019-11-12__12-30-18(2)__Oran...,2019:11:12 12:30:18,"Observer|An Nguyen, Species|Orange-headed Thrush",Panthera V4,CAM66927,Absolutely sure,An Nguyen,NaN,NaN,kpc240,ct66927,Orange-headed Thrush


## Data understanding

In [ ]:
#@title Full list of species

species_list = set([])
for s in metadata_remove_blank['species_common_name'].dropna(inplace = False).unique():
  species_list.add(s)
print(len(species_list), 'species \n')

(species_list)

In [26]:
#@title Summarize # of species total 

species_count = metadata_remove_blank['species_common_name'].value_counts()
species_count

Murid                           8411
Human                           3266
Ferret-badger                   2161
Orange-headed Thrush            1694
Bird                             985
                                ... 
Streaked Wren Babbler              1
Siberian Rubythroat                1
Golden-winged Laughingthrush       1
Indian Giant Flying Squirrel       1
Grey-shanked Douc Langur           1
Name: species_common_name, Length: 141, dtype: int64

In [38]:
#@title Summarize # of animal images uploaded

uploaded_species_counts = split_species_column[split_species_column['select'] == 'x']['species_common_name'].value_counts()

print('Uploaded:', uploaded_species_counts.sum(), 'animal images\n')
print(uploaded_species_counts)

Uploaded: 502 animal images

Owston's Civet                    131
Murid                              77
Stump-tailed Macaque               63
Human                              55
Annamite Dark Muntjac              35
Rufous-throated Partridge          29
Ferret-badger                      27
Bar-bellied Pitta                  10
Masked Palm Civet                   9
Orange-headed Thrush                8
Crab-eating Mongoose                7
Silver Pheasant                     5
Asiatic Brush-tailed Porcupine      4
Blue Whistling-thrush               4
Maxomys                             4
Asian Red-cheeked Squirrel          4
Bird                                4
Yellow-throated Marten              4
White-rumped Shama                  3
Northern Red Muntjac                3
Yellow-breasted Magpie              3
Northern Treeshrew                  2
Spotted Linsang                     2
Impressed Tortoise                  2
Crow-billed Drongo                  2
Yellow-bellied Weasel

In [39]:
#@title Record # of uploaded species & Save to species_counts.csv

species_counts = species_count.to_frame()
species_counts.rename(columns={'species_common_name':'counts'}, inplace=True)

# reset index
species_counts['species_common_name'] = species_counts.index

uploaded = []
for i in range(len(species_counts)):
  name = species_counts.index[i]
  try:
    uploaded.append(uploaded_species_counts[name])
  except:
    uploaded.append('0')

species_counts['uploaded'] = uploaded
species_counts['uploaded'] = species_counts['uploaded'].astype('int32')
species_counts['%'] = round(species_counts['uploaded']*100/species_counts['counts'], 1)
species_counts[species_counts['uploaded']>0]

# Save to file
species_counts.to_csv('species_counts.csv')
print('Saved file to species_counts.csv')
species_counts

Saved file to species_counts.csv


,counts,species_common_name,uploaded,%
Murid,8411,Murid,77,0.9
Human,3266,Human,55,1.7
Ferret-badger,2161,Ferret-badger,27,1.2
Orange-headed Thrush,1694,Orange-headed Thrush,8,0.5
Bird,985,Bird,4,0.4
...,...,...,...,...
Streaked Wren Babbler,1,Streaked Wren Babbler,0,0.0
Siberian Rubythroat,1,Siberian Rubythroat,0,0.0
Golden-winged Laughingthrush,1,Golden-winged Laughingthrush,0,0.0
Indian Giant Flying Squirrel,1,Indian Giant Flying Squirrel,0,0.0


## Preprocess **Kon Plong 2019-20 camera trap deployment_v3**

In [ ]:
YYYY-MM-DD HH:MM:SS 

## Taxon_match_table

In [41]:
import requests
import pandas as pd
import numpy as np
import json

In [42]:
#@title Download WI taxon id database

URL = "https://api.wildlifeinsights.org/api/v1/taxonomy?fields=class,order,family,genus,species,authority,taxonomyType,uniqueIdentifier,commonNameEnglish&page[size]=30000"
response = requests.get(URL)

with open('taxonomy.json', 'w') as outfile:
    json.dump(response.json()["data"], outfile)

# Save to file
taxonomy_df = pd.read_json(r'taxonomy.json')

taxonomy_df.sample(5)

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
12364,2016616,Aves,Passeriformes,Passerellidae,Melozone,occipitalis,"Salvin, 1878",Grey-crowned Ground-sparrow,biological,79c7b234-c9a8-464f-b1fe-35b4757f7eef
18513,2018558,Aves,Coraciiformes,Alcedinidae,Caridonax,None,None,Caridonax Species,biological,d1f58a91-998a-4bef-8847-11778b04355d
5144,2006816,Aves,Caprimulgiformes,Aegothelidae,Aegotheles,tatei,"Pratt, 2000",Spangled Owlet-nightjar,biological,eb9eb7d6-38b7-4a34-97eb-2ffc124ce97d
4878,2006702,Aves,Caprimulgiformes,Podargidae,Batrachostomus,affinis,"Blyth, 1847",None,biological,b70da046-5ba2-49b8-b5c4-d7bf90dca153
5603,2007427,Aves,Cuculiformes,Cuculidae,Pachycoccyx,audeberti,"Schlegel, 1879",Thick-billed Cuckoo,biological,5063bcdc-f9ad-4368-bc27-c4b375667142


In [45]:
# @title Match unfound species name with corresponding name in WI taxon id

match_unfound_species = {}
# match_unfound_species["Hill Blue Flycatcher"]
# match_unfound_species["Indochinese Wren-babbler"] = {"Taxonomy":"Gypsophila brevicaudata"}
# match_unfound_species["Black-throated Laughingthrush"] = {"Taxonomy":"Pterorhinus chinensis"}
# match_unfound_species["Banded Krait"] = {"Taxonomy":"Bungarus fasciatus"}
# match_unfound_species["Impressed Tortoise"] = {"Taxonomy":"Manouria impressa"}
# match_unfound_species["Small Asian Mongoose"] = {"genus":"Urva", "species":"javanica"}
# match_unfound_species["Natalia's Agama"] = {"commonNameEnglish":""}

# match_unfound_species["Flying Insect"] = {"commonNameEnglish":""}
# match_unfound_species["Animal"] = {"commonNameEnglish":""}

match_unfound_species["Streaked Wren Babbler"] = {"commonNameEnglish":"Streaked Wren-babbler"}
match_unfound_species["Japanese Robin"] = {"genus":"Larvivora", "species":"akahige"}
match_unfound_species["Ochraceous Bulbul"] = {"genus":"Alophoixus", "species":"ochraceus"}
match_unfound_species["Asian Water Monitor"] = {"commonNameEnglish":"Common Water Monitor"}
match_unfound_species["Asian Red-cheeked Squirrel"] = {"commonNameEnglish":"Red-cheeked Squirrel"}
match_unfound_species["Muntjac"] = {"commonNameEnglish":"Muntiacus Species"}
match_unfound_species["Asian Black Bear"] = {"commonNameEnglish":"Asiatic Black Bear"}
match_unfound_species["Ferret-badger"] = {"commonNameEnglish":"Melogale Species"}
match_unfound_species["Rhesus Macaque"] = {"commonNameEnglish":"Rhesus Monkey"}
match_unfound_species["Murid"] = {"commonNameEnglish":"Muridae Family"}
match_unfound_species["Banded Kingfisher"] = {"genus":"Lacedo", "species":"pulchella"}
match_unfound_species["Maxomys"] = {"commonNameEnglish":"Muridae Family"}
match_unfound_species["Leopoldamys"] = {"commonNameEnglish":"Muridae Family"}
match_unfound_species["Wild Pig"] = {"commonNameEnglish":"Wild Boar"}
match_unfound_species["Crested Serpent Eagle"] = {"commonNameEnglish":"Crested Serpent-Eagle"}
match_unfound_species["Scaly Thrush"] = {"genus":"Zoothera", "species":"dauma"}
match_unfound_species["Yellow-breasted Magpie"] = {"commonNameEnglish":"Indochinese Green Magpie"}
match_unfound_species["Macaque"] = {"commonNameEnglish":"Macaca Species"}
match_unfound_species["Snake"] = {"commonNameEnglish":"Lizards and Snakes"}
match_unfound_species["Annamite Dark Muntjac"] = {"commonNameEnglish":"Annamite Muntjac"}
match_unfound_species["White-Crowned Forktail"] = {"commonNameEnglish":"White-crowned Forktail"}
match_unfound_species["Particolored Flying Squirrel"] = {"commonNameEnglish":"African Linsang"}
match_unfound_species["Oriental Magpie Robin"] = {"commonNameEnglish":"Oriental Magpie-Robin"}
match_unfound_species["Squirrel"] = {"commonNameEnglish":"Sciuridae Family"}
match_unfound_species["Butterfly"] = {"commonNameEnglish":"Butterflies and Moths"}
match_unfound_species["Moth"] = {"commonNameEnglish":"Butterflies and Moths"}
match_unfound_species["Asian Palm Civet"] = {"commonNameEnglish":"Common Palm Civet"}
match_unfound_species["Shrew"] = {"commonNameEnglish":"Soricidae Family"}
match_unfound_species["White-throated Fantail"] = {"genus":"Rhipidura", "species":"albicollis"}
match_unfound_species["Racket-tailed Treepie"] = {"commonNameEnglish":"Racquet-tailed Treepie"}
match_unfound_species["Brownish-flanked Bush Warbler"] = {"genus":"Horornis", "species":"fortipes"}
match_unfound_species["Emerald Dove"] = {"commonNameEnglish":"Common Emerald Dove"}
match_unfound_species["Domestic Buffalo"] = {"commonNameEnglish":"Bubalus Species"}
match_unfound_species["Malayan Night Heron"] = {"commonNameEnglish":"Malay Night-heron"}
match_unfound_species["Lesser Mouse-deer"] = {"commonNameEnglish":"Lesser Oriental Chevrotain"}
match_unfound_species["Greater Racquet-tailed Drongo"] = {"genus":"Dicrurus", 'species':'paradiseus'}
match_unfound_species["Eastern Striped Squirrel"] = {"commonNameEnglish":"Maritime Striped Squirrel"}
match_unfound_species["Green magpie"] = {"commonNameEnglish":"Common Green Magpie"}
match_unfound_species["Flying squirrel"] = {"commonNameEnglish":"Sciuridae Family"}
match_unfound_species["White-tailed Robin"] = {"commonNameEnglish":"White-tailed Blue Robin"}
match_unfound_species["Tickell's Blue Flycatcher"] = {"commonNameEnglish":"Tickell's Blue-Flycatcher"}
match_unfound_species["Greater Racket-tailed Drongo"] = {"genus":"Dicrurus","species":"paradiseus"}
match_unfound_species["Snowy-browed Flycatcher"] = {"genus":"Ficedula", "species":"hyperythra"}
match_unfound_species["Puff-throated Bulbul"] = {"genus":"Alophoixus", "species":"pallidus"}
match_unfound_species["Pale-footed Bush Warbler"] = {"commonNameEnglish":"Pale-footed Bush-warbler"}

In [46]:
#@title Rename some species

match_unfound_species["Chinese Serow"] = {"commonNameEnglish":"Sumatran Serow"}


In [48]:
# @title Find species who is not in WI taxon db

species_list = metadata_remove_blank['species_common_name'].dropna().unique()

unfound = []

for s in species_list:
  if (s not in taxonomy_df['commonNameEnglish'].to_list()) and (s not in match_unfound_species):
    unfound.append(s)

print("There are",len(unfound),"unfound species")
(unfound)

There are 9 unfound species


['Hill Blue Flycatcher',
 'Indochinese Wren-babbler',
 'Flying Insect',
 'Black-throated Laughingthrush',
 'Animal',
 'Impressed Tortoise',
 'Banded Krait',
 'Small Asian Mongoose',
 "Natalia's Agama"]

In [44]:
taxonomy_df[taxonomy_df['species'] == 'sapiens']

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier
17499,2021525,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Camera Trapper,biological,066c4187-4a03-4531-9f62-ac585ad0a138
17500,2021526,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Hunter,biological,f5ec47a5-e7a4-4158-a0b4-a6481ae3f06f
17501,2021527,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Researcher,biological,e6f88ed1-c305-4e94-b53d-8eec1b1d980a
17502,2021528,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Tourist,biological,795880f2-2677-41bf-be13-72c58056f08d
17503,2021529,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Resident,biological,128d0fac-f96d-4a98-af18-2903dbbe1568
17545,2021530,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Park Ranger,biological,0ffb7a18-d6df-454c-a014-c99f52ec22a1
17546,2021531,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Maintenance crew,biological,41ebc3f1-ecc7-497c-acd4-6b35623a9465
17547,2021532,Mammalia,Primates,Hominidae,Homo,sapiens,None,Human-Pedestrian,biological,0de272bf-5eab-4198-a888-3a6e92f1460d
21541,2002045,Mammalia,Primates,Hominidae,Homo,sapiens,"Linnaeus, 1758",Human,biological,990ae9dd-7a59-4344-afcb-1b7b21368000


In [49]:
# @title Match species name with corresponding name in WI taxon id & Save to taxon_match_table

metadata_remove_blank = pd.read_csv('metadata_remove_blank.csv')

match_table = taxonomy_df[taxonomy_df['commonNameEnglish'] == ""]
FFI_species_name = []

species_list = metadata_remove_blank['species_common_name'].dropna().unique()

for s in species_list:
  # if s not in taxonomy_df['commonNameEnglish'].to_list():
  if (s in match_unfound_species):
    FFI_species_name.append(s)
    # if the species has commonNameEnglish
    if 'commonNameEnglish' in match_unfound_species[s]:
      if taxonomy_df[taxonomy_df['commonNameEnglish'] == match_unfound_species[s]['commonNameEnglish']].empty:
        print('Cannot find', s)
        continue
      match_table = match_table.append(taxonomy_df[taxonomy_df['commonNameEnglish'] == match_unfound_species[s]['commonNameEnglish']])
    # if the species has genus
    else:
      if taxonomy_df[(taxonomy_df['genus'] == match_unfound_species[s]['genus']) &
                      (taxonomy_df['species'] == match_unfound_species[s]['species'])].empty:
        print('Cannot find', s)
        continue
      match_table = match_table.append(taxonomy_df[(taxonomy_df['genus'] == \
                                                      match_unfound_species[s]['genus']) & \
                                                      (taxonomy_df['species'] == \
                                                      match_unfound_species[s]['species'])])

  else:
    if s in taxonomy_df['commonNameEnglish'].to_list():
      FFI_species_name.append(s)
      match_table = match_table.append(taxonomy_df[taxonomy_df['commonNameEnglish'] == s])
    else:
      pass

match_table['FFI_species_name'] = FFI_species_name

# Save to file
match_table.to_csv('taxon_match_table.csv', index = False)
print('Saved file to taxon_match_table.csv')
match_table.sample(7)

Saved file to taxon_match_table.csv


,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier,FFI_species_name
16910,2015237,Aves,Passeriformes,Muscicapidae,Cyornis,sumatrensis,"Sharpe, 1879",Indochinese Blue-flycatcher,biological,7f352b7b-094e-4c32-9682-adba9966f16c,Indochinese Blue-flycatcher
11427,2015372,Aves,Passeriformes,Muscicapidae,Ficedula,tricolor,"Hodgson, 1845",Slaty-blue Flycatcher,biological,f08c53c7-05ae-46b7-9017-f336a3e2f772,Slaty-blue Flycatcher
13148,2000801,Mammalia,Carnivora,Viverridae,Chrotogale,owstoni,"Thomas, 1912",Owston's Civet,biological,ba799fc7-6aad-455e-87a8-1f09ad5c71e4,Owston's Civet
11335,2015248,Aves,Passeriformes,Muscicapidae,Cyornis,concretus,"Müller, 1835",White-tailed Flycatcher,biological,94a2f938-b372-407b-abbd-2bf41a70a39e,White-tailed Flycatcher
18249,2018237,Mammalia,Cetartiodactyla,Cervidae,Muntiacus,None,None,Muntiacus Species,biological,a04c522f-8ca1-4c09-85b5-a0f29b8466a8,Muntjac
1913,2002543,Mammalia,Pholidota,Manidae,Manis,javanica,"Desmarest, 1822",Sunda Pangolin,biological,6cc43c63-2730-4495-8d1e-3c3b1c1b9793,Sunda Pangolin
6432,2008529,Aves,Strigiformes,Tytonidae,Phodilus,badius,"Horsfield, 1821",Oriental Bay Owl,biological,b4f26318-22be-4bdf-b4a8-b207c0ad90ef,Oriental Bay Owl
